In [20]:
import jax
import jax.numpy as jnp
import flax.linen as nn
import matplotlib.pyplot as plt
import flax
from typing import Any
import numpy as np
import functools
from einops import rearrange
import tqdm
import time
import optax

# Variational Auto-Encoders

> The **variational auto-encoder** is a generative model that works by compressing then decompressing data.
